In [245]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from bs4 import BeautifulSoup
from pathlib import Path
import re

In [246]:
from tqdm.notebook import tqdm_notebook as tqdm

In [247]:
import numpy as np

In [248]:

def access(href):
    url = "https://planetrulers.com/{}" 
    return url.format(href)

In [249]:
with open("main.html", 'r') as f:
    html_parse = BeautifulSoup(f)

In [250]:
country_age_dict = {}

In [252]:
country_age_dictd

{'Abkhazia (Republic of Abkhazia)': 'born 6 april 1963'}

In [251]:
all_entries = html_parse.find_all('a', href=True, title=True)

# Initialize the Chrome driver
chrome_options = Options()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(options=chrome_options)

pattern = 'born.*?[0-9]{4}'

problems = []

for entry in tqdm(all_entries):
    country = entry.text
    href = entry['href']
    link = access(href)

    # If the country is already in country_age_dict, pass over it
    if country in country_age_dict or country in problems:
        continue

    # Open the website
    driver.get(link)
    temp_html = driver.page_source
    temp_text = BeautifulSoup(temp_html).find_all('p')
    temp_text = [text.text for text in temp_text]
    x1 = re.findall(pattern, "".join(temp_text).lower())
    if len(x1) != 0:
        age = x1[0]
        country_age_dict[country] = age
    else:
        problems.append(link)

    
driver.quit()

country_age_dict

  0%|          | 0/204 [00:00<?, ?it/s]

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)


In [ ]:
df

,Text,Length,Birthyear
Abkhazia (Republic of Abkhazia),born 6 april 1963,17.0,1963.0
Afghanistan,born in 1949,12.0,1949.0
Albania,born 20 march 1967,18.0,1967.0
Algeria,born 17 november 1945,21.0,1945.0
Angola,born 5 march 1954,17.0,1954.0
Antigua and Barbuda,"born july 26, 1944",18.0,1944.0
Argentina,born 2 april 1959,17.0,1959.0
Armenia,born 26 july 1959,17.0,1959.0
Austria,born 18 october 1972,20.0,1972.0
Azerbaijan,"born in december 24, 1961",25.0,1961.0


In [ ]:
df = pd.DataFrame.from_dict(country_age_dict, orient='index')
df.columns = ['Text']
df['Length'] = df['Text'].apply(lambda x: len(x))
df.sort_values('Length', ascending=False)

,Text,Length
Ireland,"born into humble circumstances in limerick, an...",3381
Sudan,"born in the town of hoshe bannaga, 100 km nort...",580
Jamaica,"born in wood hall, st. catherine and attended ...",396
Portugal,"born in lisbon, he is the eldest son of baltas...",351
Eritrea,born in asmara eritrea in a little village. is...,301
...,...,...
Dominica,born 1948,9
Marshall Islands,born 1951,9
Micronesia,born 1964,9
Somaliland,born 1952,9


In [ ]:
cutoff = df['Length'].quantile(0.9)


In [ ]:
def parse_byear(text):
    if len(text) <= cutoff:
        return int(text[-4:])
    else:
        return -1

In [ ]:
df['Birthyear'] = df['Text'].apply(parse_byear)
df.query("Birthyear == -1")

,Text,Length,Birthyear
Bulgaria,"born in gotse delchev, rosen plevneliev studie...",142,-1
Canada,born in ottawa and attended collège jean-de-br...,73,-1
Egypt,"born in gamaleya, old cairo, sisi graduated fr...",267,-1
Eritrea,born in asmara eritrea in a little village. is...,301,-1
Gabon,"born albert-bernard bongo, december 30, 1935",44,-1
Indonesia,"born in pacitan, east java, on 9 september 1949",47,-1
Ireland,"born into humble circumstances in limerick, an...",3381,-1
Jamaica,"born in wood hall, st. catherine and attended ...",396,-1
Lesotho,born at scott hospital in morija on july 17 1963,48,-1
Morocco,"born in rabat on wednesday, august 21, 1963",43,-1


In [ ]:
def explore(country):
    return df.loc[country]['Text']

In [ ]:
def add_byear(country, byear):
    df.loc[country, "Birthyear"] = byear

In [ ]:
# Manually appending the errors
add_byear("Bulgaria", 1963)
add_byear("Canada", 1971)
add_byear("Egypt", 1954)
add_byear("Eritrea", 1946)
add_byear("Gabon", 1935)
add_byear("Indonesia", 1961)
add_byear("Ireland", 1941)
add_byear("Jamaica", 1945)
add_byear('Lesotho', 1963)
add_byear("Morocco", 1961)
add_byear('Nepal', 1944)
add_byear('Nicaragua', 1945)
add_byear("Portugal", 1948)
add_byear('Saint Kitts and Nevis', 1976)
add_byear('Saint Lucia', 1954)
add_byear('São Tomé and Príncipe', 1959)
add_byear("Sudan", 1951)
add_byear("Tunisia", 1958)

In [297]:
df = df.set_index("Country")

In [ ]:
problems

['https://planetrulers.com//andorra-bishop-of-urgell',
 'https://planetrulers.com//australia-governor-general',
 'https://planetrulers.com//bhutan-king',
 'https://planetrulers.com//equatorial-guinea-president',
 'https://planetrulers.com//luxembourg-prime-minister',
 'https://planetrulers.com//mauritus-president',
 'https://planetrulers.com//netherlands-queen',
 'https://planetrulers.com//oman-prime-minister',
 'https://planetrulers.com//sint-maarten-prime-minister/',
 'https://planetrulers.com//samoa-prime-minister',
 'https://planetrulers.com//san-marino-co-head',
 'https://planetrulers.com//solomon-islands-prime-minister',
 'https://planetrulers.com//syria-president',
 'https://planetrulers.com//tajikistan-president']

In [345]:
add_byear("Andorra", 1949)
add_byear("Australia", 1953)
add_byear("Bhutan", 1980)
add_byear("Equatorial Guinea", 1942)
add_byear("Luxembourg", 1963)
add_byear("Mauritus", 1959)
add_byear("Netherlands", 1967) 
add_byear("Oman", 1955)
add_byear("Sint Maarten", 1968)
add_byear("Samoa", 1957)
add_byear("San Marino", 1972)
add_byear("Solomon Islands", 1955)
add_byear("Syria", 1965)
add_byear("tajikistan", 1952)
df = df.rename(index={"Nepal,": "Nepal"})
add_byear("Nepal", 1944)

In [346]:
df['Age'] = df['Birthyear'].apply(lambda x: np.floor(2024-x))
df.sort_values("Age")

,Text,Length,Birthyear,Age,Gender
Country,,,,,
Chile,born 11 february 1986,21.0,1986.0,38.0,Male
Chad,born 1 january 1984,19.0,1984.0,40.0,Male
Mali,born c. 1983,12.0,1983.0,41.0,Male
Korea (North),born 8 january 1983,19.0,1983.0,41.0,Male
Kosovo,born 17 may 1982,16.0,1982.0,42.0,Male
Turkmenistan,born 22 september 1981,22.0,1981.0,43.0,Male
Burkina Faso,born january 1981,17.0,1981.0,43.0,Male
El Salvador,born 24 july 1981,17.0,1981.0,43.0,Male
New Zealand,born 26 july 1980,17.0,1980.0,44.0,Male


In [347]:
df['Gender'] = "Male"

In [348]:
text = """Italy
Hungary
Sweden
Barbados
Honduras
Tanzania
Moldova
Kosovo
Taiwan (Republic of China)
Slovakia
Georgia
Trinidad and Tobago
Germany
New Zealand
Estonia
Nepal,
Malta
Sint Maarten
Finland
Saint Lucia"""

In [349]:
female_presidents = text.split("\n")

In [350]:
a2 = df.query("Country in @female_presidents").index

In [351]:
[a for a in female_presidents if a not in a2]

['Nepal,']

In [352]:
add_byear("Estonia", 1973)
add_byear("Bangladesh", 1949)

In [353]:
df = df[df.index.notna()]

In [368]:
df

,Text,Length,Birthyear,Age,Gender
Country,,,,,
Abkhazia (Republic of Abkhazia),born 6 april 1963,17.0,1963.0,61.0,Male
Afghanistan,born in 1949,12.0,1949.0,75.0,Male
Albania,born 20 march 1967,18.0,1967.0,57.0,Male
Algeria,born 17 november 1945,21.0,1945.0,79.0,Male
Angola,born 5 march 1954,17.0,1954.0,70.0,Male
Antigua and Barbuda,"born july 26, 1944",18.0,1944.0,80.0,Male
Argentina,born 2 april 1959,17.0,1959.0,65.0,Male
Armenia,born 26 july 1959,17.0,1959.0,65.0,Male
Austria,born 18 october 1972,20.0,1972.0,52.0,Male


In [370]:
df.loc[np.isin(df.index, female_presidents), "Gender"] = "Female"

In [376]:
df[["Age", "Gender"]].sort_index().to_csv("data.csv")